### Part 4

In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
%matplotlib inline

In [24]:
data = loadmat("data/ex3data1.mat")

In [25]:
data

{'X': array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]]),
 '__globals__': [],
 '__header__': 'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 'y': array([[10],
        [10],
        [10],
        ..., 
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [26]:
data['X'].shape, data['y'].shape

((5000L, 400L), (5000L, 1L))

In [27]:
def sigmoid(z):  
    return 1 / (1 + np.exp(-z))

In [28]:
def cost(theta, X, y, learningRate):  
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    first = np.multiply(-y, np.log(sigmoid(X * theta.T)))
    second = np.multiply((1 - y), np.log(1 - sigmoid(X * theta.T)))
    reg = (learningRate / 2 * len(X)) * np.sum(np.power(theta[:,1:theta.shape[1]], 2))
    return np.sum(first - second) / (len(X)) + reg

In [29]:
def gradient(theta, X, y, learningRate):  
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)

    parameters = int(theta.ravel().shape[1])
    error = sigmoid(X * theta.T) - y

    grad = ((X.T * error) / len(X)).T + ((learningRate / len(X)) * theta)

    # intercept gradient is not regularized
    grad[0, 0] = np.sum(np.multiply(error, X[:,0])) / len(X)

    return np.array(grad).ravel()

In [30]:
from scipy.optimize import minimize

def one_vs_all(X, y, num_labels, learning_rate):  
    rows = X.shape[0]
    params = X.shape[1]

    # k X (n + 1) array for the parameters of each of the k classifiers
    all_theta = np.zeros((num_labels, params + 1))

    # insert a column of ones at the beginning for the intercept term
    X = np.insert(X, 0, values=np.ones(rows), axis=1)

    # labels are 1-indexed instead of 0-indexed
    for i in range(1, num_labels + 1):
        theta = np.zeros(params + 1)
        y_i = np.array([1 if label == i else 0 for label in y])
        y_i = np.reshape(y_i, (rows, 1))

        # minimize the objective function
        fmin = minimize(fun=cost, x0=theta, args=(X, y_i, learning_rate), method='TNC', jac=gradient)
        all_theta[i-1,:] = fmin.x

    return all_theta

In [31]:
rows = data['X'].shape[0]
params = data['X'].shape[1]

In [32]:
rows

5000L

In [33]:
all_theta = np.zeros((10, params + 1))

In [34]:
X = np.insert(data['X'], 0, values=np.ones(rows), axis=1)

In [35]:
theta = np.zeros(params + 1)

In [36]:
y_0 = np.array([1 if label == 0 else 0 for label in data['y']])  
y_0 = np.reshape(y_0, (rows, 1))

In [37]:
X.shape, y_0.shape, theta.shape, all_theta.shape

((5000L, 401L), (5000L, 1L), (401L,), (10L, 401L))

In [38]:
np.unique(data['y'])

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=uint8)

In [39]:
all_theta = one_vs_all(data['X'], data['y'], 10, 1)
all_theta

C:\Program Files\Anaconda3\envs\py27\lib\site-packages\ipykernel\__main__.py:6: RuntimeWarning: divide by zero encountered in log
C:\Program Files\Anaconda3\envs\py27\lib\site-packages\ipykernel\__main__.py:6: RuntimeWarning: invalid value encountered in multiply
C:\Program Files\Anaconda3\envs\py27\lib\site-packages\ipykernel\__main__.py:7: RuntimeWarning: invalid value encountered in power
C:\Program Files\Anaconda3\envs\py27\lib\site-packages\ipykernel\__main__.py:2: RuntimeWarning: overflow encountered in exp
  from ipykernel import kernelapp as app
C:\Program Files\Anaconda3\envs\py27\lib\site-packages\ipykernel\__main__.py:5: RuntimeWarning: divide by zero encountered in log
C:\Program Files\Anaconda3\envs\py27\lib\site-packages\ipykernel\__main__.py:5: RuntimeWarning: invalid value encountered in multiply


array([[ -4.67940237e+00,   0.00000000e+00,   0.00000000e+00, ...,
          9.55784384e-03,   2.89286163e-07,   0.00000000e+00],
       [ -5.75187029e+00,   0.00000000e+00,   0.00000000e+00, ...,
          6.20324117e-02,  -6.60834548e-03,   0.00000000e+00],
       [ -8.75955657e+00,   0.00000000e+00,   0.00000000e+00, ...,
         -2.68021723e-04,  -1.23029116e-06,   0.00000000e+00],
       ..., 
       [ -1.26806595e+01,   0.00000000e+00,   0.00000000e+00, ...,
         -5.60208843e+00,   6.46781209e-01,   0.00000000e+00],
       [ -8.68407259e+00,   0.00000000e+00,   0.00000000e+00, ...,
         -2.13085827e-01,   1.17478937e-02,   0.00000000e+00],
       [ -1.28023065e+01,   0.00000000e+00,   0.00000000e+00, ...,
          1.07760110e-03,   5.78891565e-05,   0.00000000e+00]])

In [40]:
def predict_all(X, all_theta):  
    rows = X.shape[0]
    params = X.shape[1]
    num_labels = all_theta.shape[0]

    # same as before, insert ones to match the shape
    X = np.insert(X, 0, values=np.ones(rows), axis=1)

    # convert to matrices
    X = np.matrix(X)
    all_theta = np.matrix(all_theta)

    # compute the class probability for each class on each training instance
    h = sigmoid(X * all_theta.T)

    # create array of the index with the maximum probability
    h_argmax = np.argmax(h, axis=1)

    # because our array was zero-indexed we need to add one for the true label prediction
    h_argmax = h_argmax + 1

    return h_argmax

In [41]:
y_pred = predict_all(data['X'], all_theta)  
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, data['y'])]  
accuracy = (sum(map(int, correct)) / float(len(correct)))  
print('accuracy = {0}%'.format(accuracy * 100))

accuracy = 97.58%


In [45]:
sum(map(float,correct))

4879.0

Better in online tutorial...and better with python 2.7

In [ ]:
# Testing sklearn

In [55]:
from sklearn import linear_model

In [57]:
logistic_classifier = linear_model.LogisticRegression(C=100.0)

In [59]:
logistic_classifier.fit(data['X'], data['y'])

C:\Program Files\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=100.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [61]:
logistic_classifier.score(data['X'], data['y'])

0.97440000000000004